# Content Based Filtering
Author: Group 22

## Warning: You sometimes need to upgrade the version of numpy if this code doesn't work

In [7]:
 pip install numpy --upgrade

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Import Libraries

In [8]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations
from tqdm import tqdm


In [9]:
ratings = "ratings.dat"
users = "users.dat"
movies = "movies.dat"

## Read Dataset

In [10]:
df_rating = pd.read_csv(
    ratings,
    sep="::",
    header=None, 
    names=['user_id', 'movie_id', 'rating', 'timestamp']
)


/tmp/ipykernel_886012/4077109094.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_rating = pd.read_csv(


In [11]:
df_user = pd.read_csv(
    users,
    sep="::",
    header=None, 
    names=['user_id', 'gender', 'age', 'occupation', 'zipcode']
)


/tmp/ipykernel_886012/2253289803.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_user = pd.read_csv(


In [12]:
df_movie = pd.read_csv(
    movies, 
    sep='::', 
    header=None, 
    names=['movie_id', 'title', 'genre'], 
    encoding = "ISO-8859-1")

/tmp/ipykernel_886012/484467727.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_movie = pd.read_csv(


In [13]:
df_rating.head(10)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [14]:
df_rating.shape

(1000209, 4)

In [15]:
df_movie.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


## Data Preprocessing

In [16]:
actual_movie_ratings = df_rating.pivot(index='movie_id', columns='user_id', values='rating')

In [17]:
actual_movie_ratings.head(10)

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,4.0,NaN,4.0,5.0,5.0,...,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,2.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
actual_movie_ratings.shape

(3706, 6040)

## Creating a tdidf matrix based on movie genres

In [19]:
tf = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4)
                                             for c in combinations(s.split('|'), r=i)))
tfidf_matrix = tf.fit_transform(df_movie['genre'])


In [20]:
tfidf_matrix

<3883x353 sparse matrix of type '<class 'numpy.float64'>'
	with 10729 stored elements in Compressed Sparse Row format>

In [21]:
tfidf_matrix.shape

(3883, 353)

In [22]:
cosine_sim = cosine_similarity(tfidf_matrix)


In [23]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=df_movie['title'], columns=df_movie['title'])

In [24]:
cosine_sim_df

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Bamboozled (2000),Bootmen (2000),Digimon: The Movie (2000),Get Carter (2000),Get Carter (1971),Meet the Parents (2000),Requiem for a Dream (2000),Tigerland (2000),Two Family House (2000),"Contender, The (2000)"
title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),1.000000,0.082508,0.067993,0.077516,0.171093,0.000000,0.067993,0.154340,0.0,0.000000,...,0.171093,0.077516,0.336438,0.000000,0.000000,0.171093,0.000000,0.000000,0.000000,0.000000
Jumanji (1995),0.082508,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.534583,0.0,0.082905,...,0.000000,0.000000,0.278515,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Grumpier Old Men (1995),0.067993,0.000000,1.000000,0.180048,0.397402,0.000000,1.000000,0.000000,0.0,0.000000,...,0.397402,0.180048,0.000000,0.000000,0.000000,0.397402,0.000000,0.000000,0.000000,0.000000
Waiting to Exhale (1995),0.077516,0.000000,0.180048,1.000000,0.453064,0.000000,0.180048,0.000000,0.0,0.000000,...,0.453064,1.000000,0.000000,0.066577,0.000000,0.453064,0.392756,0.392756,0.392756,0.127511
Father of the Bride Part II (1995),0.171093,0.000000,0.397402,0.453064,1.000000,0.000000,0.397402,0.000000,0.0,0.000000,...,1.000000,0.453064,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Meet the Parents (2000),0.171093,0.000000,0.397402,0.453064,1.000000,0.000000,0.397402,0.000000,0.0,0.000000,...,1.000000,0.453064,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
Requiem for a Dream (2000),0.000000,0.000000,0.000000,0.392756,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.392756,0.000000,0.169513,0.000000,0.000000,1.000000,1.000000,1.000000,0.324657
Tigerland (2000),0.000000,0.000000,0.000000,0.392756,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.392756,0.000000,0.169513,0.000000,0.000000,1.000000,1.000000,1.000000,0.324657


In [25]:
cosine_sim_df.shape

(3883, 3883)

In [26]:
cosine_sim_df.index

Index(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       'Waiting to Exhale (1995)', 'Father of the Bride Part II (1995)',
       'Heat (1995)', 'Sabrina (1995)', 'Tom and Huck (1995)',
       'Sudden Death (1995)', 'GoldenEye (1995)',
       ...
       'Bamboozled (2000)', 'Bootmen (2000)', 'Digimon: The Movie (2000)',
       'Get Carter (2000)', 'Get Carter (1971)', 'Meet the Parents (2000)',
       'Requiem for a Dream (2000)', 'Tigerland (2000)',
       'Two Family House (2000)', 'Contender, The (2000)'],
      dtype='object', name='title', length=3883)

In [27]:
cosine_sim_df.columns

Index(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       'Waiting to Exhale (1995)', 'Father of the Bride Part II (1995)',
       'Heat (1995)', 'Sabrina (1995)', 'Tom and Huck (1995)',
       'Sudden Death (1995)', 'GoldenEye (1995)',
       ...
       'Bamboozled (2000)', 'Bootmen (2000)', 'Digimon: The Movie (2000)',
       'Get Carter (2000)', 'Get Carter (1971)', 'Meet the Parents (2000)',
       'Requiem for a Dream (2000)', 'Tigerland (2000)',
       'Two Family House (2000)', 'Contender, The (2000)'],
      dtype='object', name='title', length=3883)

In [28]:
# Use random number generator to create training and testing dataset.
def create_test_movie_ratings(x):
    if pd.isna(x):
        return np.nan
    else:
        if np.random.rand() < 0.8:
            return x
        else:
            return np.nan

In [29]:
test_movie_ratings = actual_movie_ratings.applymap(create_test_movie_ratings)

In [30]:
test_movie_ratings

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,5.0,5.0,...,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
train_movie_ratings = test_movie_ratings.copy(deep=True)

In [32]:
train_movie_ratings

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,5.0,5.0,...,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
actual_movie_ratings

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,4.0,NaN,4.0,5.0,5.0,...,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Getting top 10 recommended movies list for Toy Story 1 (Children's and Animation Genres)

In [38]:
def genre_recommendations(i, M, items, k):
    """
    Recommends movies based on a similarity dataframe

    Parameters
    ----------
    i : str
        Movie (index of the similarity dataframe)
    M : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    items : pd.DataFrame
        Contains both the title and some other features used to define similarity
    k : int
        Amount of recommendations to return

    """
    ix = M.loc[:,i].to_numpy().argpartition(range(-1,-k,-1))
    closest = M.columns[ix[-1:-(k+2):-1]]
    closest = closest.drop(i, errors='ignore')
    return pd.DataFrame(closest, columns=["title"]).head(k)

In [42]:
df_movie[df_movie['title'].eq('Toy Story (1995)')]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy


In [43]:
rec_list = genre_recommendations('Toy Story (1995)', cosine_sim_df, df_movie[['title', 'genre']], k=10)

In [44]:
rec_list

,title
0,Aladdin and the King of Thieves (1996)
1,"Bug's Life, A (1998)"
2,Toy Story 2 (1999)
3,"American Tail: Fievel Goes West, An (1991)"
4,"American Tail, An (1986)"
5,"Rugrats Movie, The (1998)"
6,Saludos Amigos (1943)
7,"Adventures of Rocky and Bullwinkle, The (2000)"
8,Chicken Run (2000)
9,Steamboat Willie (1940)


In [67]:
df_movie[df_movie['title'].eq('Black Cauldron, The (1985)')]

,movie_id,title,genre
1964,2033,"Black Cauldron, The (1985)",Animation|Children's


In [70]:
df_movie[df_movie['title'].eq('Toy Story (1995)')]


,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy


## Finding all the missing ratings in testing dataset

In [31]:
scores1 = []

In [32]:
# Compute recommendations for each user/movie pair
for movie_id in tqdm(test_movie_ratings.index):
    title = id_to_title[movie_id]
    rec_df = rec_scores[title]
    for user_id in test_movie_ratings.columns:
        if pd.isna(test_movie_ratings.loc[movie_id, user_id]) and not pd.isna(actual_movie_ratings.loc[movie_id, user_id]):
            ratings = []
            similarities = []
            rating_similarity_pair = []
            for t, s in rec_df.items():
                m_id = df_movie.loc[df_movie["title"] == t, 'movie_id'].iloc[0]
                if (m_id not in train_movie_ratings.index):
                    continue
                rating = train_movie_ratings.loc[m_id, user_id]
                if(not pd.isna(rating)):
                    ratings.append(rating)
                    similarities.append(s)
                    rating_similarity_pair.append((rating, s))
                if len(ratings) == 5:
                    break
            numerator = np.dot(np.array([pair_[0] for pair_ in rating_similarity_pair]), np.array([pair_[1] for pair_ in rating_similarity_pair]))
            denominator = sum(similarities)
            if (denominator > 0):
                predicted = numerator / denominator
                actual = actual_movie_ratings.loc[movie_id, user_id]
                scores1.append((predicted, actual))

100%|██████████| 3706/3706 [6:25:57<00:00,  6.25s/it]   


In [34]:
print(scores1[:100])

[(3.0542830816719904, 3.0), (4.5169491875935215, 5.0), (3.814022927141504, 5.0), (3.6, 4.0), (3.286181085887839, 4.0), (3.5575083175067106, 3.0), (4.432469065005471, 5.0), (4.064462711273508, 5.0), (3.9466300298363257, 4.0), (3.87696939832057, 2.0), (4.339350407025609, 5.0), (4.382779183185253, 5.0), (4.00135631926444, 4.0), (4.463157753854373, 5.0), (4.081172662513678, 5.0), (3.9625615138243573, 5.0), (3.2, 5.0), (5.000000000000001, 5.0), (3.600000000000001, 4.0), (3.3518788334460887, 5.0), (3.9203433399989143, 5.0), (4.566192163171453, 4.0), (3.5855176768849426, 4.0), (4.155084679261766, 4.0), (2.418588145099533, 4.0), (4.29880370629489, 4.0), (4.258743600454094, 5.0), (3.3079903347704196, 4.0), (4.012624093991773, 5.0), (4.764110410883258, 5.0), (4.216925508335834, 5.0), (4.468039290722686, 5.0), (2.3345743180952043, 2.0), (3.536097900105294, 4.0), (3.57660985564639, 3.0), (4.216234532502735, 5.0), (3.2434693661665883, 4.0), (4.088343760793781, 4.0), (4.672333174436268, 5.0), (4.546

## Calculating RMSE and MAE based on scores

In [37]:
import math

In [38]:
def calculate_rmse_and_mae(predictions):
    n = len(predictions)
    rmse = math.sqrt(sum([(p[0]-p[1])**2 for p in predictions])/n)
    mae = sum([abs(p[0]-p[1]) for p in predictions])/n
    return rmse, mae

In [39]:
rsme, mae = calculate_rmse_and_mae(scores1)


In [40]:
print("RSME: ", rsme)
print("MAE: ", mae)

RSME:  1.0773752483796206
MAE:  0.8457910745108963


## Conclusion:

### In conclusion, our content based collaborative filtering yielded a score of 1.08 RMSE and 0.84 MAE which are worse than the item-to-item and user-to-user collaborative filtering